In [1]:
from __future__ import division, print_function, unicode_literals

# data process pkg
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit

# plotting pkg
import matplotlib.pyplot as plt
import seaborn as sns

# standardize features
from sklearn.preprocessing import StandardScaler

%matplotlib inline

# Preparation

In [2]:
# Load Data
df_transaction = pd.read_csv("https://s3.amazonaws.com/pernalonga/transaction_table.csv",header=0)
df_product = pd.read_csv("https://s3.amazonaws.com/pernalonga/product_table.csv",header=0)

In [3]:
# found 8 instances with negative paid amount
df_transaction[df_transaction.tran_prod_paid_amt<0]  

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
647735,93409897,2.017080e+18,2017-08-03,340,357541011,CT,0.55,1.0,-1.10,1,-0.55,0.55
6816009,73479594,2.016100e+18,2016-10-05,341,999241421,CT,16.90,1.0,-16.98,2,-0.08,16.90
6816010,73479594,2.016100e+18,2016-10-05,342,999241421,CT,33.80,2.0,-33.96,2,-0.16,16.90
7730846,40099908,2.016052e+18,2016-05-22,344,999250092,CT,1.59,1.0,-1.60,1,-0.01,1.59
9135107,51749812,2.016092e+18,2016-09-19,325,999264989,CT,0.30,1.0,-0.49,3,-0.19,0.30
10758296,42509966,2.016062e+18,2016-06-23,984,999295518,CT,3.59,1.0,-3.94,1,-0.35,3.59
17069555,16339676,2.016033e+18,2016-03-26,576,999436833,KG,5.49,1.0,-6.90,1,-1.41,5.49
18424578,7869780,2.016082e+18,2016-08-17,988,999476721,CT,3.29,1.0,-3.30,2,-0.01,3.29


In [458]:
len(df_transaction.cust_id.unique())

7920

In [4]:
# Drop transactions with negative paid amount
df_transaction=df_transaction.loc[df_transaction['tran_prod_paid_amt']>=0]

In [5]:
# Check for missing values in transaction table
sum(df_transaction.isnull().any())

0

In [6]:
# Check for missing values in product table
sum(df_product.isnull().any())

0

In [7]:
# merge df_product and df_transaction into a large big table
product=pd.merge(df_product,df_transaction,how='inner', on='prod_id')

In [8]:
# Shape of the large merged table
product.shape

(29617067, 18)

In [9]:
# First few rows of lage merged table
product.head()

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
0,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH,139662,2.017110e+18,2017-11-03,584,CT,2.89,4.0,0.00,0,2.89,0.7225
1,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH,799924,2.017111e+18,2017-11-12,349,CT,2.89,4.0,-1.45,1,1.44,0.7225
2,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH,1399898,2.017102e+18,2017-10-21,684,CT,2.89,4.0,-1.45,1,1.44,0.7225
3,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH,1399898,2.017111e+18,2017-11-11,684,CT,2.89,4.0,-1.45,1,1.44,0.7225
4,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH,1399898,2.017121e+18,2017-12-05,684,CT,2.89,4.0,-1.45,1,1.44,0.7225


# Find subcategory with least sales and at least 5 products

In [10]:
# Subset the product with only private label brands
private = product.loc[product.brand_desc == "PRIVATE LABEL", ]

In [302]:
# Find sales and number of products for each subcategory under private label brand
sub = private.groupby('subcategory_id').agg({
    'tran_prod_paid_amt':'sum',
    'prod_id':'nunique'})

In [303]:
# First few rows of subcategory sales table
sub.head()

,tran_prod_paid_amt,prod_id
subcategory_id,,
90410,5670.17,1
90414,1046.18,1
90415,2063.46,1
90416,1438.94,2
90417,10300.00,2


In [334]:
# Sort values in the subcategory sales table in ascending order
# Finidng first 5 subcategories with least sales and no less than 5 prodcut
sub.loc[sub.prod_id >4, ].sort_values('tran_prod_paid_amt', axis=0, ascending=True).head()

,tran_prod_paid_amt,prod_id
subcategory_id,,
92464,4103.90,5
93066,4587.62,7
93464,5492.02,5
93552,5623.35,5
93425,5836.52,7


In [327]:
# Sample data from subcategory 92464
private.loc[private['subcategory_id']==92464].head(1)

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
15024012,999368016,92464,RADIADOR RESISTENCIA,95566,CLIMATIZACAO,PRIVATE LABEL,AIR CONDITIONING,26869605,2.017021e+18,2017-02-08,673,CT,44.9,1.0,-12.41,1,32.49,44.9


In [328]:
# Sample data from subcategory 93066
private.loc[private['subcategory_id']==93066].head(1)

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
8498677,999257392,93066,VELA DE ANIVERSARIO,95674,ARTIGOS DE FESTA,PRIVATE LABEL,PARTY ARTICLES,39856,2.017030e+18,2017-03-03,244,CT,0.49,1.0,0.0,0,0.49,0.49


In [329]:
# Sample data from subcategory 93464
private.loc[product['subcategory_id']==93464].head(1)

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
3975932,999197823,93464,SABONETE GLICERINA,95743,SABONETES,PRIVATE LABEL,SOAPS,59984,2.016072e+18,2016-07-21,179,CT,0.44,1.0,0.0,0,0.44,0.44


In [330]:
# Sample data from subcategory 93552
private.loc[private['subcategory_id']==93552].head(1)

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
3765006,999195768,93552,ESFREGAO FIBRA NORMA,95764,ESFREGOES,PRIVATE LABEL,SCRUBS,559804,2.017020e+18,2017-02-04,387,CT,0.99,1.0,0.0,0,0.99,0.99


In [331]:
# Sample data from subcategory 93425
private.loc[private['subcategory_id']==93425].head(1)

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng,cust_id,tran_id,tran_dt,store_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price
2162948,999176714,93425,CHAMPO COMUM,95731,CHAMPO,PRIVATE LABEL,SHAMPOO,299749,2.017063e+18,2017-06-25,617,CT,1.69,1.0,0.0,0,1.69,1.69


In [456]:
# Decide to choose subcategory 92464 and create table that only contains '92464' subcategory under private label
private_sub = private.loc[private['subcategory_id']==92464]

In [413]:
# Look at sales for each product under ‘92464’ category
prod_sales = private_sub.groupby('prod_id').agg({
    'tran_prod_paid_amt':'sum'
}).reset_index()

prod_sales['prod_id'] = prod_sales['prod_id'].astype('str')
prod_sales

,prod_id,tran_prod_paid_amt
0,999368016,646.62
1,999368017,707.59
2,999368018,1270.15
3,999436358,870.27
4,999504191,609.27


# Find customers buying least sales

In [335]:
# Find customers who bought subcategory_id 92464 under private label
cust=private.loc[private['subcategory_id']==92464].cust_id.unique()

In [336]:
# Number of customers who purchased sub-category(93419) under private labels
cust.size

170

In [338]:
# A table that consists of customers who bought subcategory_id 92464 under private label
private_product=private_sub[private_sub.cust_id.isin(cust)]
private_product['prod_id'] = private_product['prod_id'].astype('str')

In [342]:
# A table that includes customers and the products they purchased in subcategory_id 92464 under private label
prod_table = pd.DataFrame(private_product.groupby('cust_id')['prod_id'].unique().apply(list)).reset_index()
prod_table.columns = ['cust_id', 'prod_id']
prod_table['cust_id'] = prod_table['cust_id'].astype('str')
prod_table.head()

,cust_id,prod_id
0,119781,[999368018]
1,339627,[999368018]
2,2389981,[999368018]
3,3969876,[999436358]
4,5399819,[999368018]


# Recommendation to customers

In [91]:
# Read customers jaccard similarity table conducted in R
jaccard = pd.read_csv("https://s3.amazonaws.com/mltest960807/result.csv")

In [92]:
# First 5 rows of similarity table
jaccard.head(5)

,Unnamed: 0,139662,799924,1399898,1749580,1889991,1979557,2109544,2559894,2649945,...,36429965,14359520,59179784,66869557,81779569,99319674,21859962,9599639,49309956,85429551
0,139662,1.000000,0.159114,0.086903,0.136175,0.118996,0.140491,0.145704,0.143312,0.133381,...,0.064234,0.073308,0.104306,0.066540,0.080681,0.099472,0.107937,0.079199,0.085841,0.064265
1,799924,0.159114,1.000000,0.099071,0.154194,0.146295,0.137195,0.128688,0.158208,0.132336,...,0.074898,0.063636,0.078059,0.048917,0.094643,0.105232,0.098391,0.073607,0.072812,0.060519
2,1399898,0.086903,0.099071,1.000000,0.102391,0.094158,0.093770,0.107603,0.099513,0.104590,...,0.072173,0.025100,0.040282,0.029835,0.055994,0.076428,0.045939,0.054665,0.048341,0.032944
3,1749580,0.136175,0.154194,0.102391,1.000000,0.133514,0.137910,0.154128,0.142190,0.135323,...,0.087428,0.064458,0.062832,0.049372,0.060521,0.097668,0.075261,0.076987,0.069689,0.062442
4,1889991,0.118996,0.146295,0.094158,0.133514,1.000000,0.122314,0.152484,0.134241,0.108532,...,0.078444,0.061093,0.054632,0.060359,0.063636,0.092061,0.086266,0.077508,0.064394,0.048680


In [93]:
# Shape of jaccard matrix
jaccard.shape

(7920, 7921)

In [354]:
# Only include rows of customers who purchased private label such category
jaccard_cust = jaccard[jaccard.iloc[:,0].isin(cust)]

In [357]:
# First 5 rows of customers who purchased private label such category
jaccard_cust.head()

,Unnamed: 0,139662,799924,1399898,1749580,1889991,1979557,2109544,2559894,2649945,...,36429965,14359520,59179784,66869557,81779569,99319674,21859962,9599639,49309956,85429551
75,26869605,0.116833,0.120674,0.104895,0.164786,0.152231,0.107794,0.169022,0.138119,0.122330,...,0.080189,0.052330,0.050971,0.054954,0.058707,0.101648,0.079132,0.080343,0.076078,0.047377
96,33099816,0.128311,0.144566,0.102517,0.148538,0.133045,0.113450,0.155733,0.147704,0.132363,...,0.061441,0.065530,0.083107,0.059946,0.057625,0.125749,0.089191,0.090832,0.086251,0.061567
106,37449943,0.127045,0.134508,0.107570,0.128671,0.137285,0.101931,0.164188,0.168657,0.126263,...,0.074074,0.062798,0.069786,0.051917,0.062259,0.109323,0.093471,0.092917,0.088821,0.062706
211,72549599,0.137119,0.150142,0.121111,0.158818,0.151264,0.123853,0.157068,0.141372,0.126615,...,0.065523,0.053495,0.057576,0.052858,0.060496,0.111111,0.071737,0.072014,0.073200,0.048568
284,93539545,0.106223,0.139589,0.105983,0.149319,0.144609,0.128926,0.145598,0.146771,0.101548,...,0.062540,0.048819,0.056604,0.042097,0.057015,0.084868,0.078912,0.071161,0.062219,0.047347


In [358]:
# Rename first column to be cust_id and set this column as index
jaccard_cust.rename(columns={'Unnamed: 0':'cust_id'}, inplace=True)
jaccard_cust.set_index('cust_id', inplace = True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [359]:
# Replace correlation cell of 1 to 0 so that using max function to find the most similar customer other than itself
jaccard_cust[jaccard_cust == 1] = 0

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:2554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


In [361]:
# Create a new dataframe where the two columns represent a customer and another customer who is the most similar to
# cust_id is the customer we recommend products to, cust_recommend is customer who is most similar to cust_id
cust_pairs = pd.DataFrame(jaccard_cust.idxmax()).reset_index()
cust_pairs.columns = ['cust_id', 'cust_recommend']
cust_pairs['cust_recommend'] = cust_pairs['cust_recommend'].astype('str')

In [367]:
# Merge customer table with product table 
merge_table = cust_pairs.merge(prod_table, on = 'cust_id', how = "left")
recommendation = merge_table.merge(prod_table, left_on='cust_recommend', right_on='cust_id', how = "left", 
                                   suffixes=('', '_recommend'))

In [369]:
# First 5 rows of the merged table
recommendation.head(5)

,cust_id,cust_recommend,prod_id,cust_id_recommend,prod_id_recommend
0,139662,13439985,NaN,13439985,[999436358]
1,799924,23989897,NaN,23989897,[999504191]
2,1399898,9519828,NaN,9519828,[999368018]
3,1749580,55919816,NaN,55919816,[999436358]
4,1889991,97439673,NaN,97439673,[999504191]


In [380]:
# List out all the possible new products to each customer
recommendation.rename(columns={'cust_id_recommend':'recommendation'}, inplace=True)
for i in range(0,len(recommendation)):
    recommendation.recommendation.iloc[i] = list(np.setdiff1d(recommendation.prod_id_recommend.iloc[i],recommendation.prod_id.iloc[i]))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [457]:
# First 5 rows of recommendation table
# Recommendation column is the products that can promote to cust_id
recommendation.head()

,cust_id,cust_recommend,prod_id,recommendation,prod_id_recommend
0,139662,13439985,NaN,[999436358],[999436358]
1,799924,23989897,NaN,[999504191],[999504191]
2,1399898,9519828,NaN,[999368018],[999368018]
3,1749580,55919816,NaN,[999436358],[999436358]
4,1889991,97439673,NaN,[999504191],[999504191]


In [483]:
# Change recommendation table format into one row per customer per recommended product
# Since customers can be recommended more than one products
finalprod = recommendation.recommendation.\
apply(pd.Series).merge(recommendation.iloc[:,[0,3]], left_index = True, right_index = True).\
drop(["recommendation"], axis = 1).\
melt(id_vars = ['cust_id'], value_name = "prod_id").\
drop(["variable"], axis = 1)
finalprod.dropna(inplace=True)

In [484]:
# Example of cust_id who is recommened for more than 1 product
finalprod.loc[finalprod.cust_id == '30309997',]

,cust_id,prod_id
7898,30309997,999368018
15818,30309997,999504191


In [435]:
# Merge recommendation table to product sales table to 
finalprodsales = finalprod.merge(prod_sales, on = 'prod_id', how = "left")

In [452]:
# Only recommend each customer the product with the higher sales
final_rec = finalprodsales.groupby('cust_id').max().reset_index().iloc[:,0:2]
final_rec.head()

,cust_id,prod_id
0,10009638,999504191
1,10029739,999368018
2,10079850,999368018
3,1009831,999504191
4,1009995,999504191


In [486]:
# Sample matched above where we select only one sales
final_rec.loc[final_rec.cust_id == '30309997',]

,cust_id,prod_id
1798,30309997,999504191


In [455]:
# We recommend products to 7860 customers, excluding 60 poeple who have already purchased several products
final_rec.shape

(7860, 2)

In [507]:
# Recommendation frequency for each product under this subcategory
prod_count = pd.DataFrame(final_rec.prod_id.value_counts()).reset_index()
prod_count.columns = ['prod_id', 'count']
prod_count

,prod_id,count
0,999368018,3860
1,999504191,1822
2,999436358,1021
3,999368017,1017
4,999368016,140
